In [2]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [8]:
classifier = Sequential()

classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

classifier.add(Conv2D(32, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

classifier.add(Conv2D(32, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

classifier.add(Flatten())
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=3, activation='softmax'))

classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
train_data_dir = './images/train'
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
training_set = train_datagen.flow_from_directory('/Volumes/data/Yi/2020Spring/295B/Online-training-system/server/media/training/classifiedImage/training/',
                                                 target_size=(64, 64),
                                                 batch_size=16,
                                                 class_mode='categorical')
print(training_set)
nb_train_samples = len(training_set.filenames)
print(nb_train_samples)
num_classes = len(training_set.class_indices)
print(num_classes)

Found 138 images belonging to 3 classes.
138
3


In [13]:
test_data_dir = './images/test'
test_datagen = ImageDataGenerator(rescale=1. / 255)
test_set = test_datagen.flow_from_directory('/Volumes/data/Yi/2020Spring/295B/Online-training-system/server/media/training/classifiedImage/testing/',
                                            target_size=(64, 64),
                                            batch_size=16,
                                            class_mode='categorical')
print(test_set)
nb_test_samples = len(test_set.filenames) 
print(nb_test_samples)
num_classes = len(test_set.class_indices)
print(num_classes)

Found 12 images belonging to 3 classes.
12
3


In [14]:
classifier.fit_generator(training_set,
                         steps_per_epoch=138,
                         epochs=1,
                         validation_data=test_set,
                         validation_steps=12)

Epoch 1/1
138/138 [==============================] - 48s 349ms/step - loss: 0.9443 - accuracy: 0.5236 - val_loss: 1.0270 - val_accuracy: 0.5833


In [15]:
import numpy as np
from keras.preprocessing import image

In [17]:
test_image = image.load_img('/Volumes/data/Yi/2020Spring/295B/Online-training-system/server/media/training/classifiedImage/testing/Beagle/3098-beagle.jpg',
                            target_size =(64,64))

In [18]:
test_image = image.img_to_array(test_image)

In [19]:
test_image = np.expand_dims(test_image, axis = 0)

In [21]:
result = classifier.predict(test_image)

In [22]:
result

array([[1., 0., 0.]], dtype=float32)

In [23]:
training_set.class_indices

{'Beagle': 0, 'Cat': 1, 'GoldenRetriever': 2}

In [24]:
classifier.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 12, 12, 32)        9248      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1152)             

In [40]:
classifier.save('keras.h5') 

In [46]:
import tensorflow as tf
model=tf.keras.models.load_model("keras.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

In [49]:
open("converted_model.tflite", "wb").write(tflite_model)

672432